In [ ]:
!sudo apt install build-essential

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!pip install cmake cython
!pip install networkit

     |████████████████████████████████| 3.2 MB 32.2 MB/s 
  Created wheel for networkit: filename=networkit-9.0-cp37-cp37m-linux_x86_64.whl size=8338076 sha256=1d7e177ea01589959947f3061629c34087eaa889f796d633d961a9cca73e0abb
  Stored in directory: /root/.cache/pip/wheels/b7/de/25/080d79ccb8820c024decc38731f5575b6790191e7b79ed87db
Successfully built networkit


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import networkit as nk
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
data_path = "/content/drive/MyDrive/Diplomamunka/Link prediction/data"
os.listdir(data_path)

['networkit.ipynb', 'mooc', 'lastfm', 'reddit', 'wikipedia', 'cee.ipynb']

In [ ]:
def predict(predictor, u,i):
  try:
    return predictor.run(labels[str(int(u))], labels[str(int(i))])
  except:
    return np.nan

In [ ]:
for folder in os.listdir(data_path):
  if os.path.isdir(os.path.join(data_path,folder)):
    data_name = folder
    print(data_name)

    if os.path.isfile(os.path.join(data_path, f"static_result_{data_name}.csv")):
      continue
    
    label_path = os.path.join(data_path, f"{data_name}/train/{data_name}_train.json")
    graph_path = os.path.join(data_path, f"{data_name}/train/{data_name}_train.gml")
    test_path = os.path.join(data_path, f"{data_name}/test/ml_{data_name}.csv")

    with open(label_path, 'r') as fp:
        labels = json.load(fp)
    test = pd.read_csv(test_path, index_col=0)
    G = nk.readGraph(graph_path, nk.Format.GML)

    pai = nk.linkprediction.PreferentialAttachmentIndex(G)
    tqdm.pandas(desc="Calculate PreferentialAttachmentIndex...")
    test['pa'] = test.progress_apply(lambda x: predict(pai, x.u, x.i), axis=1)
    # test.pa.fillna(0, inplace=True)

    nm = nk.linkprediction.NeighborsMeasureIndex(G)
    tqdm.pandas(desc="Calculate NeighborsMeasureIndex...")
    test['nm'] = test.progress_apply(lambda x: predict(nm, x.u, x.i), axis=1)
    # test.nm.fillna(0, inplace=True)

    katz = nk.linkprediction.KatzIndex(G)
    tqdm.pandas(desc="Calculate KatzIndex...")
    test['ra'] = test.progress_apply(lambda x: predict(katz, x.u, x.i), axis=1)
    # test.ra.fillna(0, inplace=True)

    dist = nk.linkprediction.AlgebraicDistanceIndex(G, 30, 200)
    dist.preprocess()
    tqdm.pandas(desc="Calculate AlgebraicDistanceIndex...")
    test['dist'] = test.progress_apply(lambda x: predict(dist, x.u, x.i), axis=1)
    # test.dist.fillna(0, inplace=True)

    test.to_csv(os.path.join(data_path, f"static_result_{data_name}.csv"))

mooc


Calculate KatzIndex...:  97%|█████████▋| 160221/164702 [22:05<00:27, 162.75it/s]